#### 2023-09-07(수)구글스토어 퀘스트

In [11]:
from selenium import webdriver

In [12]:
# open Chrome browser.executable_path = 브라우저 여는 법 
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [13]:
#'https://www.w3schools.com/' 윈도우 창에 특정 주소 연결 
# url in address window
# enter 치면 새창 생성./#html css가 local로 가져옴 
browser.get('https://play.google.com/store/search?q=%ED%97%AC%EC%8A%A4%EC%BC%80%EC%96%B4%EC%95%B1&c=apps&hl=ko-KR')

 ### 서비스 리스트 가져오기 
 - section > div > div > div > div:nth-child(2) > div > div > div

### 2023-09-07(목)추가 적인 내용 특정 서비스 클릭 하기 

In [14]:
browser.implicitly_wait(10)

In [15]:
import pandas as pd  #판다스 만들기 
import pymongo as mg # 몽고db 만들기 
import time

### 선생님이 작성한 코드 (이걸로 실행해야함 )

In [16]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['googlestore_heathcare']

# 서비스 앱에 대한 리스트
click_path = 'section > div > div > div > div'
company_list = browser.find_elements_by_css_selector(click_path)

len(company_list)

for company in company_list:
    company.click()
    time.sleep(2)

    # 서비스 앱에 대한 정보 가져오기
    try:
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except :
        service_name = str()
    try :    
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except : 
        company_name = str()
    try :    
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except :
        app_information = str()
    
    ### 리뷰 총 갯수 가져오기['리뷰 473개']
    ## 리뷰 총 갯수 tag : div:nth-child(1) > div.g1rdde
    review_total_count_text = browser.find_element_by_css_selector('div:nth-child(1) > div.g1rdde').text
    # [service_name, company_name, app_information]
    import re #regularexpression 
    result_list = re.findall(r'\d+', review_total_count_text) #규칙?r(이진수의 값을 적용하라는 말) /string 
    #print(result_list[0] ,int(result_list[0]))

    review_total_count = int(result_list[0]) # 리뷰 총 갯수가 (int)로 변환돼서 나옴. 
    
    
    # 댓글 팝업 띄우기(해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    review_flag = True
    try :
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click()    
    except :
        review_flag = False
    
    # 회사 서비스에 대한 전체 리뷰 가져오기
    serviceapp_columns_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag :
        #팝업 띄우기 
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)    
             
        #팝업 속 댓글 전체 표시 
        review_standard_count_per = 40


        loop_count_int = int(review_total_count / review_standard_count_per)
        for count in range(1, loop_count_int+1) :
            try :
                reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
                print('current reviews_bundle count : {}'.format(len(reviews_bundle)))
                reviews_bundle[len(reviews_bundle)-1].click()
                time.sleep(3)
            except :
                print('pass')
                pass
        print('Done', len(reviews_bundle))
        
        
        #팝업 댓글 수집 
        for review_bundle in reviews_bundle:
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list)  # 리뷰에 대한 모든 것(특정 회사 서비스에 대한)

        browser.back() # 팝업 닫기
        
        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
    browser.back() # 리스트로 돌아가기
    time.sleep(3)
print('Done')

current reviews_bundle count : 40
current reviews_bundle count : 80
current reviews_bundle count : 120
Done 120
service_name : Samsung Health(삼성 헬스), review count : 120
current reviews_bundle count : 40
current reviews_bundle count : 80
current reviews_bundle count : 120
current reviews_bundle count : 160
current reviews_bundle count : 200
current reviews_bundle count : 240
current reviews_bundle count : 248
current reviews_bundle count : 248
current reviews_bundle count : 248
current reviews_bundle count : 248
current reviews_bundle count : 248
Done 248
service_name : NH헬스케어, review count : 248
Done 17
service_name : 케어헬스, review count : 17
Done 7
service_name : 기업용 디지털 헬스케어 삼성화재 애니핏 프로(PRO), review count : 7
Done 40
service_name : 애니핏 플러스 나에게 핏한 건강관리, review count : 40
Done 2
service_name : SK VIEW 헬스케어, review count : 2
Done 14
service_name : 헬스케치-건강검진 기반 건강지표/위험도 분석,맞춤 헬스, review count : 14
Done 1
service_name : 헬스케어 건강관리서비스, review count : 1
Done 24
service_name : PT쌤 - 운동 헬스 체중관리

IndexError: list index out of range

In [ ]:
broswer.quit()

### MJ가 작성한 코드 (오류) 주석만 참고.  

In [ ]:
#서비스 앱에 대한 리스트 가져오기. 
click_path = 'section > div > div > div > div'
company_list = browser.find_elements_by_css_selector(click_path)

len(company_list)


### 해당되는 회사 클릭 회사 서비스에 대한 전체 리뷰 가져오기 

for company in company_list:
    company.click()
    time.sleep(2)
    
     # 서비스 앱에 대한 정보 가져오기
    try:
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except :
        service_name = str()
    try :    
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except : 
        company_name = str()
    try :    
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except :
        app_information = str()

    # [service_name, company_name, app_information]
    
   # 댓글 팝업 띄우기(해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    review_flag = True
    try :
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click()    
    except :
        review_flag = False
    
        
    # 회사서비스에 대한 전체 리뷰 reviews_bundle 로 가져오기 
    serviceapp_columns_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag :
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)    

        #reviews_list 가져오기 40개만 looping 
         for review_bundle in reviews_bundle:
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list)  # 리뷰에 대한 모든 것(특정 회사 서비스에 대한)

        browser.back() # 팝업 닫기
        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name)
    
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
    browser.back() # 리스트로 돌아가기
    time.sleep(3)

####  제품정보
- 서비스명 :NH 헬스케어 
    -  div > h1 > span  
- 만든회사명 tag 
    - div.Il7kR > div > div > div > a > span
- 앱정보 tag 
    - div:nth-child(1) > c-wiz:nth-child(2) > div > section > div > div.bARER